In [149]:
import re
import pandas as pd

In [168]:
with open("_chat.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [169]:
print(data)


‎[11/6/24, 3:09:04 PM] ~ Moiz Mirza: ‎image omitted
[11/6/24, 9:33:40 PM] ‪+1 (678) 824‑3014‬: ‎‪+1 (678) 824‑3014‬ left
[11/9/24, 4:17:30 PM] ‪+91 82508 29267‬: ‎‪+91 82508 29267‬ joined from the community
[11/9/24, 4:17:53 PM] ‪+91 82508 29267‬: ‎‪+91 82508 29267‬ left
[11/11/24, 4:17:29 AM] ~ ~: ‎‎~ ~ changed their phone number to a new number. ‎Tap to message or add the new number.
[11/12/24, 11:34:37 AM] ‪+92 331 8070938‬: ‎‪+92 331 8070938‬ left
[11/18/24, 11:43:14 AM] ~ Max: ‎~ Max joined from the community
[11/18/24, 11:43:39 AM] ~ Max: ‎~ Max left
[11/18/24, 8:21:17 PM] ~ 😁: ‎~ 😁 joined using this group's invite link
[11/19/24, 8:51:46 PM] ‪+92 305 2029478‬: ‎‪+92 305 2029478‬ left
[11/20/24, 1:10:52 AM] ‪+92 334 3191934‬: ‎‪+92 334 3191934‬ left
[11/21/24, 5:47:16 PM] ~ Sk: ‎~ Sk joined from the community
[11/22/24, 12:48:04 PM] ~ Muzamil: Hey. How much time do you take to review form ?
[11/28/24, 1:54:16 PM] ~ Mariyam: ‎~ Mariyam left
[11/29/24, 10:14:13 PM] Sahil Bachani D

In [171]:
pattern = r"\[\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}:\d{2}\s?[APMapm]{2}\]"

In [172]:
dates = re.findall(pattern, data)

In [173]:
messages = re.split(pattern, data)[1:]

In [174]:
print("Found dates:", len(dates))
print("Found messages:", len(messages))


Found dates: 586
Found messages: 586


In [175]:
if len(dates) != len(messages):
    print("⚠️ Mismatch between dates and messages!")
else:
    df = pd.DataFrame({"message_date": dates, "user_message": messages})
    print(df.head())

             message_date                                       user_message
0   [11/6/24, 3:09:04 PM]                     ~ Moiz Mirza: ‎image omitted\n
1   [11/6/24, 9:33:40 PM]   ‪+1 (678) 824‑3014‬: ‎‪+1 (678) 824‑3014‬ left\n
2   [11/9/24, 4:17:30 PM]   ‪+91 82508 29267‬: ‎‪+91 82508 29267‬ joined ...
3   [11/9/24, 4:17:53 PM]       ‪+91 82508 29267‬: ‎‪+91 82508 29267‬ left\n
4  [11/11/24, 4:17:29 AM]   ~ ~: ‎‎~ ~ changed their phone number to a ne...


In [176]:
df = pd.DataFrame({
    "message_date": dates,
    "user_message": messages
})

In [177]:
df["message_date"] = df["message_date"].str.replace(r"[\[\]\u202F]", "", regex=True)

In [178]:
print(df["message_date"].head())
print(df["message_date"].isna().sum(), "unparsed rows")

0     11/6/24, 3:09:04PM
1     11/6/24, 9:33:40PM
2     11/9/24, 4:17:30PM
3     11/9/24, 4:17:53PM
4    11/11/24, 4:17:29AM
Name: message_date, dtype: object
0 unparsed rows


In [179]:
df["message_date"] = pd.to_datetime(
    df["message_date"], 
    format="%m/%d/%y, %I:%M:%S%p",  # no space before %p
    errors="coerce"
)

In [180]:
users, msgs = [], []
for m in df["user_message"]:
    entry = re.split(r"([\w\W]+?):\s", m, maxsplit=1)
    if len(entry) > 2:
        users.append(entry[1])
        msgs.append(entry[2])
    else:
        users.append("group_notification")
        msgs.append(entry[0])

df["user"] = users
df["message"] = msgs
df.drop(columns=["user_message"], inplace=True)

In [181]:
df["date"] = df["message_date"].dt.date
df["year"] = df["message_date"].dt.year
df["month"] = df["message_date"].dt.month_name()
df["day"] = df["message_date"].dt.day
df["day_name"] = df["message_date"].dt.day_name()
df["hour"] = df["message_date"].dt.hour
df["minute"] = df["message_date"].dt.minute

In [182]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   message_date  586 non-null    datetime64[ns]
 1   user          586 non-null    object        
 2   message       586 non-null    object        
 3   date          586 non-null    object        
 4   year          586 non-null    int32         
 5   month         586 non-null    object        
 6   day           586 non-null    int32         
 7   day_name      586 non-null    object        
 8   hour          586 non-null    int32         
 9   minute        586 non-null    int32         
dtypes: datetime64[ns](1), int32(4), object(5)
memory usage: 36.8+ KB
None


,message_date,user,message,date,year,month,day,day_name,hour,minute
0,2024-11-06 15:09:04,~ Moiz Mirza,‎image omitted\n,2024-11-06,2024,November,6,Wednesday,15,9
1,2024-11-06 21:33:40,‪+1 (678) 824‑3014‬,‎‪+1 (678) 824‑3014‬ left\n,2024-11-06,2024,November,6,Wednesday,21,33
2,2024-11-09 16:17:30,‪+91 82508 29267‬,‎‪+91 82508 29267‬ joined from the community\n,2024-11-09,2024,November,9,Saturday,16,17
3,2024-11-09 16:17:53,‪+91 82508 29267‬,‎‪+91 82508 29267‬ left\n,2024-11-09,2024,November,9,Saturday,16,17
4,2024-11-11 04:17:29,~ ~,‎‎~ ~ changed their phone number to a new numb...,2024-11-11,2024,November,11,Monday,4,17
